In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [1]:
import pickle

with open('URM_train_validation.pkl', 'rb') as f:
    URM_train_validation = pickle.load(f)
with open('URM_test.pkl', 'rb') as f:
    URM_test = pickle.load(f)

In [2]:
from Hybrid import *
from Recommenders.SLIM.SLIMElasticNetRecommender import *
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
## SLIM
slim = SLIMElasticNetRecommender(URM_train_validation)
slim.fit(topK=8894, l1_ratio=0.05565733019999427, alpha=0.0012979360257937668)#, workers=7)
## RP3BETA
rp3beta = RP3betaRecommender(URM_train_validation)
rp3beta.fit(alpha=0.20026352123406477, beta=0.15999879728761354, topK=32)
## ITEM KNN
itemknn = ItemKNNCFRecommender(URM_train_validation)
itemknn.fit(topK=31, shrink=435, similarity='tversky', normalize=True,
                                       feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)
from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])

SLIMElasticNetRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 109 ( 0.5%) items with no interactions.
SLIMElasticNetRecommender: Processed 22057 (99.3%) in 5.00 min. Items per second: 73.50
SLIMElasticNetRecommender: Processed 22222 (100.0%) in 5.06 min. Items per second: 73.13
RP3betaRecommender: URM Detected 205 ( 1.6%) users with no interactions.
RP3betaRecommender: URM Detected 109 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 7438.56 column/sec. Elapsed time 2.99 sec
ItemKNNCFRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 109 ( 0.5%) items with no interactions.
Similarity column 22222 (100.0%), 10447.06 column/sec. Elapsed time 2.13 sec
EvaluatorHoldout: Ignoring 2203 (17.4%) Users that have less than 1 test interactions


In [11]:
hybrid = TwoScoresHybridRecommender(URM_train_validation, slim, rp3beta)

TwoScoresHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
TwoScoresHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.


In [12]:
import optuna as op
def objective(trial):
    alpha = trial.suggest_float('alpha', 0, 1)
    
    # Create hybrid recommender with the suggested weights
    hybrid.fit(alpha=alpha)
    
    # Evaluate the hybrid recommender
    result, _ = evaluator_validation.evaluateRecommender(hybrid)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [15]:
best_params = {
    "alpha": 1,
}
study_name = "hybrid-slim-rp3-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
#op.delete_study(study_name, storage_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
study.enqueue_trial(best_params)
study.optimize(objective, n_trials=100)

[I 2023-12-30 00:42:29,821] Using an existing study with name 'hybrid-slim-rp3-study' instead of creating a new one.


EvaluatorHoldout: Processed 10435 (100.0%) in 4.40 sec. Users per second: 2370


[I 2023-12-30 00:42:34,276] Trial 300 finished with value: 0.0506210098797551 and parameters: {'alpha': 1.0}. Best is trial 190 with value: 0.051900719495592995.


EvaluatorHoldout: Processed 10435 (100.0%) in 4.18 sec. Users per second: 2498


[I 2023-12-30 00:42:38,477] Trial 301 finished with value: 0.051887554095268996 and parameters: {'alpha': 0.5914978770686858}. Best is trial 190 with value: 0.051900719495592995.


...

[I 2023-12-30 00:49:31,553] Trial 397 finished with value: 0.051416044903827035 and parameters: {'alpha': 0.36986067972124764}. Best is trial 378 with value: 0.05190429035373978.


EvaluatorHoldout: Processed 10435 (100.0%) in 4.27 sec. Users per second: 2445


[I 2023-12-30 00:49:35,844] Trial 398 finished with value: 0.05182210737673168 and parameters: {'alpha': 0.5956131476475066}. Best is trial 378 with value: 0.05190429035373978.


EvaluatorHoldout: Processed 10435 (100.0%) in 4.25 sec. Users per second: 2455


[I 2023-12-30 00:49:40,118] Trial 399 finished with value: 0.051684688281957616 and parameters: {'alpha': 0.6788738274199106}. Best is trial 378 with value: 0.05190429035373978.


In [16]:
study.best_params

{'alpha': 0.8857667100747117,
 'beta': 0.6379807942443021,
 'gamma': 0.0690929184825888,
 'delta': 0.11953478623354052}